In [1]:
#! apt-get update -y

In [2]:
#! apt-get install wget -y

In [3]:
#! apt-get install unzip -y

In [4]:
#! wget LoanStats_web.csv.zips://storage.googleapis.com/19sep/LoanStats_web.csv.zip

In [5]:
#! unzip LoanStats_web.csv.zip

In [6]:
from pyspark.sql import functions as sparkf

In [7]:
from pyspark.sql import SparkSession

# สร้าง SparkSession ใหม่
spark = (
    SparkSession.builder
        # กำหนดชื่อแอปพลิเคชัน
        .appName("Integration of onlinePassenger and Station")
        # กำหนด URL ของ Spark master
        .master("spark://spark-master:7077")
        # กำหนดจำนวน memory ที่ executor จะใช้
        .config("spark.executor.memory", "1000m")
        # กำหนดจำนวน cores ที่ executor จะใช้
        .config("spark.executor.cores", "2")
        # กำหนดจำนวน cores สูงสุดที่ Spark จะใช้
        .config("spark.cores.max", "6")
        # สร้าง SparkSession ถ้ายังไม่มี, ถ้ามีแล้วจะใช้ SparkSession ที่มีอยู่
        .config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector:1.0.2") \
        .config("spark.sql.debug.maxToStringFields", 200) \

        .getOrCreate()
)

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.azure#spark-mssql-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a7bf1ac1-d074-46ed-9bcf-e89c64d57440;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector;1.0.2 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 97ms :: artifacts dl 2ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector;1.0.2 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnl

In [8]:
# ใช้ SparkSession ที่เราสร้างขึ้น (spark) ในการอ่านข้อมูล

loanStat_raw_df = (
    spark.read  # ใช้วิธีการอ่านข้อมูล (read method)
        .format("csv")  # กำหนดรูปแบบของไฟล์ที่จะอ่านเป็น CSV
        .option("header", "true")  # กำหนดว่าไฟล์ CSV มี header ที่บรรทัดแรก
        .load("LoanStats_web.csv")  # โหลดข้อมูลจากไฟล์
)

In [9]:
loanStat_raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [10]:
from pyspark.sql.functions import when, col, regexp_extract, regexp_replace

# กำหนดรูปแบบ regex สำหรับการตรวจสอบ
term_pattern = "^(36|60)$"
int_rate_pattern = "^([0-9]+[.]?[0-9]*|)$"  # ตัวเลขทั่วไป หรือ ตัวเลขที่มีจุดทศนิยม
grade_pattern = "^[A-G]$"

# ทำการ validation
valid_df = loanStat_raw_df.withColumn(
    "term_valid", 
    when(col("term").isNull(), "ไม่สามารถประเมินได้")
    .otherwise(
        when(regexp_extract(col("term"), term_pattern, 1) != "", "True")
        .otherwise("False")
    )
).withColumn(
    "int_rate_valid", 
    when(col("int_rate").isNull(), "ไม่สามารถประเมินได้")
    .otherwise(
        when(
            (regexp_extract(col("int_rate"), int_rate_pattern, 1) != "") &
            (regexp_replace(col("int_rate"), "%", "").cast("double") >= 0) & 
            (regexp_replace(col("int_rate"), "%", "").cast("double") <= 30), "True"
        ).otherwise("False")
    )
).withColumn(
    "grade_valid", 
    when(col("grade").isNull(), "ไม่สามารถประเมินได้")
    .otherwise(
        when(regexp_extract(col("grade"), grade_pattern, 0) != "", "True")
        .otherwise("False")
    )
)

# แสดงผล
valid_df.select("term", "term_valid", "int_rate", "int_rate_valid", "grade", "grade_valid").show()


+----------+----------+--------+--------------+-----+-----------+
|      term|term_valid|int_rate|int_rate_valid|grade|grade_valid|
+----------+----------+--------+--------------+-----+-----------+
| 36 months|     False|   9.75%|         False|    B|       True|
| 36 months|     False|   5.32%|         False|    A|       True|
| 36 months|     False|   7.39%|         False|    A|       True|
| 36 months|     False|  11.47%|         False|    B|       True|
| 36 months|     False|   5.32%|         False|    A|       True|
| 36 months|     False|   7.39%|         False|    A|       True|
| 36 months|     False|  11.99%|         False|    C|       True|
| 36 months|     False|  14.46%|         False|    C|       True|
| 36 months|     False|  11.99%|         False|    C|       True|
| 36 months|     False|   9.75%|         False|    B|       True|
| 36 months|     False|  12.99%|         False|    C|       True|
| 36 months|     False|  12.99%|         False|    C|       True|
| 60 month

In [11]:
valid_df.groupBy("term", "term_valid").count().show()

+----------+-------------------+-------+
|      term|         term_valid|  count|
+----------+-------------------+-------+
|      null|ไม่สามารถประเมินได้|     26|
| 36 months|              False|1036235|
| 60 months|              False| 396205|
+----------+-------------------+-------+



In [12]:
valid_df.groupBy("int_rate", "int_rate_valid").count().show()

+--------+-------------------+-----+
|int_rate|     int_rate_valid|count|
+--------+-------------------+-----+
|  30.17%|              False|  438|
|  18.20%|              False|    3|
|   5.32%|              False|36953|
|  11.48%|              False| 2064|
|  25.69%|              False|  863|
|  30.94%|              False|  324|
|  10.33%|              False| 9686|
|  23.40%|              False| 4028|
|  13.19%|              False|    1|
|  17.57%|              False|   15|
|  17.86%|              False|    7|
|   9.44%|              False| 4064|
|  14.49%|              False| 8875|
|  25.65%|              False|  149|
|    null|ไม่สามารถประเมินได้|   26|
|  21.48%|              False|  356|
|  25.49%|              False| 4250|
|  18.06%|              False| 1977|
|  15.05%|              False| 3423|
|   7.35%|              False| 3699|
+--------+-------------------+-----+
only showing top 20 rows



In [13]:
valid_df.groupBy("grade", "grade_valid").count().show()

+-----+-------------------+------+
|grade|        grade_valid| count|
+-----+-------------------+------+
|    A|               True|302988|
| null|ไม่สามารถประเมินได้|    26|
|    B|               True|424110|
|    G|               True|  5271|
|    D|               True|191137|
|    E|               True| 68353|
|    C|               True|421101|
|    F|               True| 19480|
+-----+-------------------+------+



In [15]:
valid_df.filter(col('int_rate_valid')== "ไม่สามารถประเมินได้").toPandas().transpose()

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:183: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:183: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series
/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:183: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
id,Total amount funded in policy code 1: 2087217200,Total amount funded in policy code 2: 662815446,Total amount funded in policy code 1: 1443412975,Total amount funded in policy code 2: 511988838,Total amount funded in policy code 1: 1404586950,Total amount funded in policy code 2: 567447023,Total amount funded in policy code 1: 1465324575,Total amount funded in policy code 2: 521953170,Total amount funded in policy code 1: 1437969475,Total amount funded in policy code 2: 520780182,...,Total amount funded in policy code 1: 1741781700,Total amount funded in policy code 2: 564202131,Total amount funded in policy code 1: 2080429200,Total amount funded in policy code 2: 737901574,Total amount funded in policy code 1: 2063142975,Total amount funded in policy code 2: 823319310,Total amount funded in policy code 1: 2050909275,Total amount funded in policy code 2: 820109297,Total amount funded in policy code 1: 1928448350,Total amount funded in policy code 2: 799372035
member_id,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
loan_amnt,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
funded_amnt,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
funded_amnt_inv,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
settlement_percentage,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
settlement_term,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
term_valid,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,...,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้
int_rate_valid,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,...,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้,ไม่สามารถประเมินได้
